In [1]:
#%%capture
print('Starting...')
import logging
from datetime import datetime
import sys
import os
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    print('Installing dependencies older GPUs...')
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer,  StoppingCriteria, StoppingCriteriaList, GenerationConfig
from unsloth import FastLanguageModel
print('End of pip install...')

Starting...
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-oub2uhm5/unsloth_60d85f4abfa64e7692d5a5c12bbb0b69
  Running command git clone -q https://github.com/unslothai/unsloth.git /tmp/pip-install-oub2uhm5/unsloth_60d85f4abfa64e7692d5a5c12bbb0b69
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 1.3 MB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 83.6 MB/s eta 0:00:01
  Created wheel for unsloth: filename=unsloth-2024.4-py3-none-any.whl size=96655 sha256=ccebbff6be8e52c25dc0d81f6ee8bc36d44cc632cf1592ff218db6733afa4ed0
  Stored in directory: /tmp/pip-ephem-wheel-cache-qlweeub1/wheels/0b/bf/f5/61523189908a01bce8752a181f02f8b057ffc2c792447d39ff
Successfully built unsloth
  Attempting uninstall: wheel
    Found existing installation:

In [2]:
print('Starting...')
# Obtener la fecha y hora actual
fecha_hora_actual = datetime.now()

# Crear un manejador para la consola
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Formatear la fecha y hora
texto_fecha_hora = fecha_hora_actual.strftime("%Y%m%d_%H%M")
mylog = "comebien_"+texto_fecha_hora+".log"

# Crear un manejador para el archivo
file_handler = logging.FileHandler(filename=mylog, encoding = "UTF-8")
file_handler.setLevel(logging.INFO)

# Definir el formato de los registros
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
#console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Crear un logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Agregar los manejadores al logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)
logger.info('Comienzo de script')
#from huggingface_hub import notebook_login
#notebook_login()

Comienzo de script


Starting...


In [3]:
logger.info('Cargando modelos...')
#my_run = "mistral7biv02_2""
#my_run = "gemma-2b-it-bnb-4bit"
# 23/03/24 my_run = "mistral-7b-instruct-v0.2-bnb-4bit" 
# 24/03/2024 Error con my_run = "gemma-7b-it-bnb-4bit"
# 28/03/2024 Ok 60 steps "llama-2-7b-bnb-4bit"
# 28/03/24 ok 60 "mistral-7b-instruct-v0.2-bnb-4bit"
# 29/03/24 ok 2000 1h "gemma-2b-it-bnb-4bit"
#my_run = "gemma-2b-it-bnb-4bit"
enable_gemma = True
if enable_gemma:
    my_run = 'gemma-2b-bnb-4bit'
    #sft_model_name = 'somosnlp/ComeBien_DIXREC10_gemma-2b-it-bnb-4bit'
    sft_model_name = 'somosnlp/ComeBien_Nutr100_gemma-2b-bnb-4bit'
    sft_model_name = 'somosnlp/RecetasDeLaAbuela5k_gemma-2b-bnb-4bit'
else:
    my_run = 'mistral-7b-bnb-4bit'
    sft_model_name = 'somosnlp/ComeBien_DIXREC10_mistral-7b-bnb-4bit'

base_model_name = "unsloth/" + my_run

#sft_model = "somosnlp/ComeBien_mistral-7b-instruct-v0.2-bnb-4bit"
#base_model_name = "unsloth/Mistral-7B-Instruct-v0.2"
#sft_model_name = "somosnlp/ComeBien_Ing_gemma-2b-it-bnb-4bit"
#sft_model_name = "somosnlp/ComeBien_gemma-2b-it-bnb-4bit"
#sft_model_name = "somosnlp/ComeBien_URL10_gemma-2b-it-bnb-4bit"

logger.info('Usando modelo base '+base_model_name)
logger.info('Usando modelo SFT '+sft_model_name)

max_seq_length=150
#base_model = AutoModelForCausalLM.from_pretrained(base_model_name,return_dict=True,device_map="auto",torch_dtype=torch.float16,)
#tokenizer = AutoTokenizer.from_pretrained(base_model_name, max_length = max_seq_length)

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(model_name = sft_model_name,max_seq_length = max_seq_length,dtype=None, load_in_4bit = True,)

if False:
    from unsloth import FastLlamaModel, FastMistralModel
    base_model, tokenizer = FastMistralModel.from_pretrained(model_name = base_model_name,max_seq_length = max_seq_length,dtype = None,load_in_4bit = True,)
    logger.info('Modelo base cargado ' +str(base_model_name))
    ft_model = PeftModel.from_pretrained(base_model, sft_model_name, torch_dtype=torch.float16,)
    logger.info(type(ft_model))
    logger.info('Mezclando modelos...')
    model = ft_model.merge_and_unload()

logger.info(type(model))
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")
logger.info('Modelo cargado ' +str(sft_model_name))

Cargando modelos...
Usando modelo base unsloth/gemma-2b-bnb-4bit
Usando modelo SFT somosnlp/RecetasDeLaAbuela5k_gemma-2b-bnb-4bit


adapter_config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

PyTorch version 2.2.2 available.


adapter_model.safetensors:   0%|          | 0.00/78.5M [00:00<?, ?B/s]

Unsloth 2024.4 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.
<class 'peft.peft_model.PeftModelForCausalLM'>
Modelo cargado somosnlp/RecetasDeLaAbuela5k_gemma-2b-bnb-4bit


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [4]:
# Inference functions
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    #instructions = examples["instruction"]
    inputs       = examples["Nombre"]
    outputs      = examples["DIXREC"]
    texts = []
    for input, output in zip(inputs, outputs):
        instruction = 'Eres un agente experto en nutrición y cocina.'
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# alpaca_prompt = Copied from above
def generar_alpaca_respuesta(pregunta, contexto, logger, model):
    prompt = alpaca_prompt.format(contexto, pregunta, '',) + EOS_TOKEN
    logger.info(prompt)
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")    
    outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True)    
    res=tokenizer.batch_decode(outputs)    
    logger.info(res)
    return res


class ListOfTokensStoppingCriteria(StoppingCriteria):
    """
    Clase para definir un criterio de parada basado en una lista de tokens específicos.
    """
    def __init__(self, tokenizer, stop_tokens):
        self.tokenizer = tokenizer
        # Codifica cada token de parada y guarda sus IDs en una lista
        self.stop_token_ids_list = [tokenizer.encode(stop_token, add_special_tokens=False) for stop_token in stop_tokens]

    def __call__(self, input_ids, scores, **kwargs):
        # Verifica si los últimos tokens generados coinciden con alguno de los conjuntos de tokens de parada
        for stop_token_ids in self.stop_token_ids_list:
            len_stop_tokens = len(stop_token_ids)
            if len(input_ids[0]) >= len_stop_tokens:
                if input_ids[0, -len_stop_tokens:].tolist() == stop_token_ids:
                    return True
        return False

# Uso del criterio de parada personalizado
stop_tokens = ["<end_of_turn>"]  # Lista de tokens de parada

# Inicializa tu criterio de parada con el tokenizer y la lista de tokens de parada
stopping_criteria = ListOfTokensStoppingCriteria(tokenizer, stop_tokens)

# Añade tu criterio de parada a una StoppingCriteriaList
stopping_criteria_list = StoppingCriteriaList([stopping_criteria])

def generate_text(prompt, context, model, max_length=max_seq_length):
  prompt=prompt.replace("\n", "").replace("¿","").replace("?","")
  #input_text = f'''<bos><start_of_turn>system ¿{context}?<end_of_turn><start_of_turn>user ¿{prompt}?<end_of_turn><start_of_turn>model'''
  #https://unsloth.ai/blog/gemma-bugs
  enable_gemma = True
  if enable_gemma:
    input_text = f'''<bos><start_of_turn>system\n{context}<end_of_turn><start_of_turn>user\n{prompt}<end_of_turn><start_of_turn>model\n'''
  else:
    intro = 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request'
    input_text = f'''{intro}\n\n### Instruction:\n\n{context}\n\n### Input:\n{prompt}\n\n### Response: '''


  inputs = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=False).to("cuda:0")
  max_new_tokens=max_length
  generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                temperature=0.32, #top_p=0.9, top_k=50, # 45
                repetition_penalty=1.3,  #1.1 #1.04
                do_sample=True,
            )
  if enable_gemma:
    outputs = model.generate(generation_config=generation_config, input_ids=inputs, stopping_criteria=stopping_criteria_list,)
  else:
    outputs = model.generate(generation_config=generation_config, input_ids=inputs)
  return tokenizer.decode(outputs[0], skip_special_tokens=False) #True

def mostrar_respuesta(pregunta, contexto, logger, model):
    try:
        res= generate_text(pregunta, contexto, model, 700)
        print(res)
    except Exception as e:
        res = e    
    #logger.info(res.encode('utf-8'))
    logger.info(str(res))
    return res


In [5]:
 torch.cuda.is_bf16_supported()

False

In [6]:
test1 = False
if test1:
    context = 'Eres un agente experto en nutrición y cocina'     
    instruction = 'Cantidad de calorías grasas sodio fibra sal azúcar de '
    items = ['Tortilla de patatas', 'Ceviche', 'Enchiladas']
    for item in items:
        pregunta = instruction + item
        mostrar_respuesta(pregunta, context, logger, model)

In [7]:
test2 = False
if test2:
    context = "You are a technical specialist in food and cooking. Dime en español el valor nutricional de "
    mostrar_respuesta("Tortilla de patatas", context, logger, model)
    mostrar_respuesta("Ceviche", context, logger, model)
    mostrar_respuesta("Enchiladas", context, logger, model)
    context = "You are a technical specialist in food and cooking. Resume en una sola frase el valor nutricional de "
    mostrar_respuesta("Tortilla de patatas", context, logger, model)
    mostrar_respuesta("Ceviche", context, logger, model)
    mostrar_respuesta("Enchiladas", context, logger, model)

In [8]:
!pip install evaluate bert_score
from evaluate import load
bertscore = load("bertscore")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     |████████████████████████████████| 84 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 61 kB 24.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 86.8 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 106.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 96.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 111.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 304 kB 103.4 MB/s eta 0:00:01


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
generated new fontManager


In [ ]:
import pandas as pd

# Paso 1: Leer el archivo CSV en un DataFrame
archivo_csv = 'recetasabuela_es.csv'  # Reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo CSV
dft = pd.read_csv(archivo_csv)

test30 = True
if test30:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Ingredientes y pasos de la receta '    
    #recetas = ["Tortilla de patatas", "Ceviche", "Enchiladas", "Empanada de puerro y nata", "Cordero asado", "Paella", "Guiso de carne", "Potaje de garbanzos", "Callos madrileños" ]
    #recetas = ["repollo guisado"]
    #for receta in recetas:
    df = dft.head(20)
    for indice, fila in df.iterrows():        
        receta = query + fila['Nombre']        
        output = mostrar_respuesta(receta, context, logger, model)
        predictions = [receta, output]
        ref_output = 'Ingredientes: '+fila['Ingredientes']+'. Pasos: '+fila['Pasos']
        references = [receta, ref_output]
        results = bertscore.compute(predictions=predictions, references=references, lang="es")
        logger.info(results)
        df.loc[indice, 'precision'] = results['precision'][1]
        df.loc[indice, 'recall'] = results['recall'][1]
        df.loc[indice, 'f1'] = results['f1'][1]
    df.to_csv('recetasabuela_es_ds.csv', index=False)
    logger.info('Promedio precision: '+str(df['precision'].mean()))
    logger.info('Promedio recall: '+str(df['recall'].mean()))
    logger.info('Promedio f1: '+str(df['f1'].mean()))

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta tarta de pollo y verduras<end_of_turn><start_of_turn>model
Ingredientes: 1 disco de masa para tarta (aproximadamente 26 cm) ½ kilogramo de carne picada mixta pollo o mixto con vegetales cocidos al gusto tomate frito sal pimienta nuez moscada aceite virgen extra leche agua. Pasos: 0 Para empezar a preparar esta deliciosa tarta casera primero debes reunir todos los ingredientes que necesitas. 1 A continuación coloca sobre una superficie plana el disco de masa del tamaño aproximado requerido por ti puedes pedirlo directamente en tu carnicería aunque también se puede comprar ya cortado. Si no tienes este tipo de discos te proponemos hacer tú mismo uno siguiendo las instrucciones paso a paso aquí. Truco: si usas otro tipo de masa es importante tener cuidado cuando estirasla porque tiene menos espesor. 3 Después extiende ligeramente encima la base cubrien

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta tarta de pollo y verduras<end_of_turn><start_of_turn>model
Ingredientes: 1 disco de masa para tarta (aproximadamente 26 cm) ½ kilogramo de carne picada mixta pollo o mixto con vegetales cocidos al gusto tomate frito sal pimienta nuez moscada aceite virgen extra leche agua. Pasos: 0 Para empezar a preparar esta deliciosa tarta casera primero debes reunir todos los ingredientes que necesitas. 1 A continuación coloca sobre una superficie plana el disco de masa del tamaño aproximado requerido por ti puedes pedirlo directamente en tu carnicería aunque también se puede comprar ya cortado. Si no tienes este tipo de discos te proponemos hacer tú mismo uno siguiendo las instrucciones paso a paso aquí. Truco: si usas otro tipo de masa es importante tener cuidado cuando estirasla porque tiene menos espesor. 3 Después extiende ligeramente encima la base cubrien

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta pastel de pan de molde con jamon york y queso al horno<end_of_turn><start_of_turn>model
Ingredientes: 1 lámina de Masa para pizza o Pan de molde ½ kilogramo de Jamón York (o similar) picado muy fino 250 gramos de Queso mozzarella rallado 1 paquete pequeño de Tocineta ahumada picada a cuadritos Sal Aceite virgen extra 1 lata pequeña de Leche condensado Harina Para rebozar el pastel de jamón York y tocineta Almejas u otro marisco fresco para decorar si quieres ¡y aproveche! Pasos: 1 El primer paso es precalentar el horno a unos 180-200 ºC aproximadamente durante diez minutos como mínimo. Mientras se calienta la placa colocamos los trozos de bacon sobre una bandeja forrada con papel vegetal que luego sacaremos del pastel de pan de molde fácil así evitaremos cualquier tipo de contaminación cruzada por las manos ya verás cómo no te hará falta lavar nada 

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta pastel de puerros bacon y hojaldre<end_of_turn><start_of_turn>model
Ingredientes: 1 kilogramo de espinacas frescas o congeladas ½ kilo de puré de papas (o papa) 2 unidades de pimientos rojos pequeños cortados por el centro sin semillas ¼ unidad de cebolla morada cortada finamente 30 gramos de bacon cortado a tiras finas Sal gruesa Hojaldre Aceite para freír Queso rallado al gusto Pimienta negra molida Harina Perejil picado Almendra picada. Pasos: 4 Prepara todos los ingredientes antes de empezar a preparar este delicioso Pastel de Puerro con Bacon y Hojaldre. 5 Para hacer nuestro relleno primero corta las verduras muy pequeñas tal como se ve en la foto; luego calentamos una olla grande cubriéndola hasta llegar casi arriba colocamos todo dentro excepto el queso que lo haremos más adelante cuando esté caliente. Lo dejamos cocinar durante unos minutos 

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta pastel de puerros bacon y hojaldre<end_of_turn><start_of_turn>model
Ingredientes: 1 kilogramo de espinacas frescas o congeladas ½ kilo de puré de papas (o papa) 2 unidades de pimientos rojos pequeños cortados por el centro sin semillas ¼ unidad de cebolla morada cortada finamente 30 gramos de bacon cortado a tiras finas Sal gruesa Hojaldre Aceite para freír Queso rallado al gusto Pimienta negra molida Harina Perejil picado Almendra picada. Pasos: 4 Prepara todos los ingredientes antes de empezar a preparar este delicioso Pastel de Puerro con Bacon y Hojaldre. 5 Para hacer nuestro relleno primero corta las verduras muy pequeñas tal como se ve en la foto; luego calentamos una olla grande cubriéndola hasta llegar casi arriba colocamos todo dentro excepto el queso que lo haremos más adelante cuando esté caliente. Lo dejamos cocinar durante unos minutos 

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta quiche de puerros y champinones<end_of_turn><start_of_turn>model
Ingredientes: 1 lámina para quiche ½ kilogramo de espinacas o acelgas congeladas (o frescas) ¼ kilo de setas variadas al gusto 2 huevos cocidos Aceite Sal Pimienta Albahaca molida Queso rallado para gratinar. Pasos: 0 Reunir los ingredientes que vamos a utilizar es el primer paso antes de empezar con esta deliciosa quiche casera fácil. Truco: Si quieres hacer las verduras tu mismo puedes seguir estas instrucciones sencillas de cómo cocinar vegetales frescos sin necesidad de microondas ni horno. 1 A continuación se fríe ligeramente por separado todos los productos vegetal-animal como son las láminas fritas los huevos duros cocidos los pimientos verdes secos también cortados muy bien ... así no va a quedar una quiche de puerro crujiente pero demasiado líquida porque todo todavía está sec

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta quiche de puerros y champinones<end_of_turn><start_of_turn>model
Ingredientes: 1 lámina para quiche ½ kilogramo de espinacas o acelgas congeladas (o frescas) ¼ kilo de setas variadas al gusto 2 huevos cocidos Aceite Sal Pimienta Albahaca molida Queso rallado para gratinar. Pasos: 0 Reunir los ingredientes que vamos a utilizar es el primer paso antes de empezar con esta deliciosa quiche casera fácil. Truco: Si quieres hacer las verduras tu mismo puedes seguir estas instrucciones sencillas de cómo cocinar vegetales frescos sin necesidad de microondas ni horno. 1 A continuación se fríe ligeramente por separado todos los productos vegetal-animal como son las láminas fritas los huevos duros cocidos los pimientos verdes secos también cortados muy bien ... así no va a quedar una quiche de puerro crujiente pero demasiado líquida porque todo todavía está sec

In [ ]:
logger.info('Promedio precision: '+str(df['precision'].mean()))
logger.info('Promedio recall: '+str(df['recall'].mean()))
logger.info('Promedio f1: '+str(df['f1'].mean()))

In [ ]:
test31 = False
if test31:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Dime los ingredientes y pasos de la receta '
    recetas = ["Tortilla de patatas", "Ceviche", "Enchiladas", "Empanada de puerro y nata", "Cordero asado", "Paella", "Guiso de carne", "Potaje de garbanzos", "Callos madrileños" ]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)
    
    #mostrar_respuesta("¡Un delicioso milhojas de otoño!", context, logger, model)
    #mostrar_respuesta("milhojas de otoño", context, logger, model)
    #mostrar_respuesta("¡No has probado nada igual!", context, logger, model)
    #mostrar_respuesta("¡Un plato rápido y saludable!", context, logger, model)

In [ ]:
test32 = True
if test32:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Dime los ingredientes y pasos de la receta '
    recetas = ["Pollo en pepitoria", "Lombarda con piñones", "Huevos a la flamenca", "Guisantes con jamón", "Empanada de carne", "Rosbif de ternera", "Cochinillo", "Puré de verduras" ]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)
    

In [ ]:
test33 = True
if test33:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Dime los ingredientes y pasos de la receta '
    recetas = ["garbanzos criollos", "mariscada al ajillo"]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)

In [ ]:
test34 = True
if test34:
    context = "Eres un agente experto en nutrición y cocina."
    #query = 'Dime los nombres de las recetas más populares que conozcas similares a '
    query = 'Dime el valor nutricional de '
    recetas = ["arrroz con pollo", "carne de cerdo guisada"]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)

In [ ]:
test4 = False
if test4:
    context = "Eres un especialista en nutricion y cocina. Dime el valor nutricional de la "
    context = 'Eres un agente experto en nutrición y cocina. Cantidad de calorías, grasas, sodio, azúcar'
    #context = 'You are a helpful AI assistant. Eres un agente experto en nutrición y cocina. Lista de codigos linguisticos disponibles: [''es'', ''en'']'
    #mostrar_respuesta(" Dime la cantidad de grasas de la capirotada", context, logger)
    mostrar_respuesta("capirotada", context, logger, model)
    context = 'Eres un agente experto en nutrición y cocina. Cantidad de calorías, grasas, sodio, azúcar'
    #context = 'You are a helpful AI assistant. Eres un agente experto en nutrición y cocina. Lista de codigos linguisticos disponibles: [''es'', ''en'']'
    #mostrar_respuesta(" Dime la cantidad de grasas de la capirotada", context, logger)
    mostrar_respuesta("capirotada", context, logger, model)
    context = 'Eres un agente experto en nutrición y cocina'
    mostrar_respuesta("Dime la receta de la capirotada", context, logger, model)
    mostrar_respuesta("Dime la receta de los Tacos Dorados de Papa", context, logger, model)
    mostrar_respuesta("Dime la receta de Coditos con queso Amarillo y pimiento morrón", context, logger, model)
    mostrar_respuesta("Dime la receta de lomo saltado con tallarines a la huancai", context, logger, model)
    mostrar_respuesta("Dime la duración de la receta Cebollas Moradas Curtidas", context, logger, model)
    mostrar_respuesta("Dime la duración de la receta Tortilla de patatas Cebollas Moradas Curtidas", context, logger, model)

In [ ]:
test5 = False
if test5:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Cantidad de calorías, grasas, sodio, azúcar de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [ ]:
test6 = False
if test6:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'País originario de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [ ]:
test7 = False
if test7:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Tiempo de preparación de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [ ]:
test8 = False
if test8:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Grado de dificultad de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [ ]:
test9 = False
if test9:
    context = 'Eres un agente experto en nutrición y cocina'
    #instruction = 'Dime la URL de la receta '
    instruction = 'Dime la DIXREC de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [ ]:
test10 = False
if test10:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Dime la DIXREC de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        #mostrar_respuesta(item, context, logger, model)
        generar_alpaca_respuesta(item, context, logger, model, model)

In [ ]:
# alpaca_prompt = Copied from above
test11 = False
if test11: 
    #import time
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    instruction = 'Eres un agente experto en nutrición y cocina.'
    output = ''
    inputs_list = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    input = 'Tacos Dorados de Papa'
    input = 'Rajas de Chile Poblano con Crema'

    #for input in inputs_list:
    #    print(input)
    inputs = tokenizer([alpaca_prompt.format(instruction, input, output,)], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    tokenizer.batch_decode(outputs)
    #time.sleep(5)